<h2>How many products falls into customer budget along with list of products?In case of clash, choose the less costly product
</h2>

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Product-Budget-Analysis').getOrCreate()

In [2]:
from pyspark.sql.functions import to_date

data = [('P1',200),('P2',300),('P3',500),('P4',800)] 
schema = """product_id string,
cost int"""
product_df=spark.createDataFrame(data,schema)
product_df.printSchema()

data=[(100,400),(200,800),(300,1500)]
schema="customer_id int,budget int"
customer_budget_df=spark.createDataFrame(data,schema)
customer_budget_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- cost: integer (nullable = true)

root
 |-- customer_id: integer (nullable = true)
 |-- budget: integer (nullable = true)



In [3]:
from pyspark.sql.window import Window

window_spec=Window.orderBy(product_df.cost.asc())

In [19]:
from pyspark.sql.functions import sum,col,collect_list,concat_ws

product_df_cum=product_df.withColumn("cum_cost",sum(product_df.cost).over(window_spec))

final_df=customer_budget_df.join(product_df_cum).filter(product_df_cum.cum_cost<=customer_budget_df.budget).groupBy(customer_budget_df.customer_id,customer_budget_df.budget).agg(collect_list(product_df_cum.product_id).alias("products"))
final_df_int=final_df.collect()
x=[]
for i in final_df_int:
    x.append({
    "customer_id":i['customer_id'],
    "budget":i['budget'],
    "products":','.join(i['products'])
    })
data=[]
for i in x:
    data.append(i)

final_df=spark.createDataFrame(data,"customer_id int ,budget int,products string")
final_df.show()

+-----------+------+--------+
|customer_id|budget|products|
+-----------+------+--------+
|        100|   400|      P1|
|        200|   800|   P1,P2|
|        300|  1500|P1,P2,P3|
+-----------+------+--------+

